In [16]:
'''This program is designed to create a 3d graph relating s and n to E[M_n] from a .json file. of the type
where s is not fixed, as in stemplate.json. It is not restricted to the circle alone, but is general.'''

# This cell is composed of the necessary imports and class formations for the program.

# Used for plotting data:
import matplotlib.pyplot as plt

# Used to read outside files:
from msgspec.json import decode
from msgspec import Struct
from typing import Dict

# The following classes allows for specifying a path for decoidng the .json file in get_data below:

class Max(Struct): 
    meanOfMaxima: float # Assigns to Max an attribute 'meanOfMaxima' of type float.

class Sval(Struct): 
    s: Dict[str, Max] # Assigns to Sval an attribute 's' of type Dict from str to Max.

class Info(Struct):
    n: Dict[str, Sval] # Assigns to Info an attribute 'n' of type Dict from str to Sval.

class Access(Struct):
    info: Info # Assigns to Access an attribute 'info' of type Info.

In [17]:
'''This cell reads and decodes a .json file to efficiently access its information. 
getData takes in a .json file of the type where s is not fixed, as in stemplate.json and only 
decodes the part of the file whose class is Access. Access has an attribute called 
'info' of type Info, which has an attribute 'n' that is a dictionary whose keys are strings, 
specifically the available nvalues, and whose values are of Sval, and Sval has an attribute
's' that is a dictionary whose keys are strings, specifically the available svalues, and 
whose values are of type Max, which has the lone attribute meanOfMaxima of type float.
This method allows only the paths we want: info > n > str(nval) > s > str(sval) > meanOfMaxima > mean.'''

def getData(file: str) -> Access: # getData accepts .json files where s is not fixed.
    with open(file, 'rb') as f:
         return decode(f.read(), type = Access)
        
'''createSbynGraph takes in one argument: data = get_data(file), and collects each pair (n, s, mean)
in a given path into lists of nvals, svals, and mvals that can be zipped to prroduce a 3D-graph with 
n on the x-axis, s on the y-axis, and the associated meanOfMaxima on the z-axis.''''' 
        

def createSByNGraph(data: Access) -> None:
    nvals = [] # To be plotted on the x-axis.
    svals = [] # To be plotted on the y-axis.
    mvals = [] # To be plotted on the z-axis
    for n_key in data.info.n.keys(): # This path will iterate over all values of n.
        for s_key in data.info.n[n_key].s.keys(): # This path will iterate over all values of s under n_key.
            mean = data.info.n[n_key].s[s_key].meanOfMaxima # This path will give the mean for the given n and s.
            # Appends the data to the val lists, ensuring that nvals are ints and svals are floats, not strings:
            nvals.append(int(n_key)) 
            svals.append(float(s_key))
            mvals.append(mean)
    ax = plt.axes(projection = '3d') # Creates the 3D-plot axes.
    # Labels the axes accordingly:
    ax.set_xlabel('n')
    ax.set_ylabel('s')
    ax.set_zlabel('E[Mn]')
    # Creates the scatter plot with colors according to the mvals, cmap 'coolwarm', and opacity alpha = 1:
    ax.scatter3D(nvals, svals, mvals, c = mvals, cmap='coolwarm', alpha=1)